# Affective Computing - Programming Assignment 1




#### Objective

The objective of the exercise is to build a facial expression recognition system. The system includes face preprocessing, feature extraction and classification. In the exercise, you will learn how to preprocess a facial expression image, extract features from an image or a video, and classify a video into a category.

Specifically, the region of interest (i.e., facial image) is extracted using face tracking, face registration and face crop functions.  Basic spatiotemporal features (i.e., LBP-TOP features) are extracted using LBP-TOP. For classificaiton of the extracted features Support Vector Machine (SVM) classifiers are trained. 50 videos from 5 participants are used to train the emotion recognition, using spatiotemporal features. The rest of the data (50 videos) are used to evaluate the performances of the trained recognition system.

#### Database
The original facial expression data is a sub-set of eNTERFACE (acted facial expression), from ten actors acting **happy** and **sadness** behaviors. The used dataset in the exercise includes 100 facial expression samples. 

<!---#### Help-->
<!--- The data and toolbox files used in this exercise can be found in the Affective Computing course webpage (see the Noppa system).
-->

<!---In the exercise, you should know that some basic python libries (numpy, scikit-image, scipy, pyploy, sklearn) before the programming. If you have questions, send your question to us: 
* [Henglin.Shi@oulu.fi](mailto:Henglin.Shi@oulu.fi)
* [Yante.Li@oulu.fi](mail.Yante.Li@oulu.fi)-->

<!---Use the following website to help the usage of python libries. 
* numpy: http://www.numpy.org/
* skimage: https://scikit-image.org/
* scipy: https://www.scipy.org/
* matplotlib.pyplot: https://matplotlib.org/api/pyplot_api.html
* sklearn: http://scikit-learn.org/stable/
    * sklearn.svm: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
    * skealrn.metrics.confusion_matrix: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html-->

## Task 1. Face preprocessing

In this part, you are supposed to process a face image. There are three subtasks you need to do:
* **Task 1.1.** Detect face and facial landmarks using the [`DLib`](http://dlib.net/) library.
* **Task 1.2.** You are asked to perform a face registration task using a set of fixed landmarks from a standard model, and extract face from the registered image.
* **Task 1.3.** Visualize your result using subplots.

### Task 1.1. Extract facial landmarks <font color='red'>(1 point)</font>
**Steps**:
1. Load the example image *example_img.jpg*, using the [`plt.imread()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html) function for example.
2. As a sanity check visualize the image to see what you are working with.
3. Initialize a face detector and a face landmarks detector. We have provide the code of this part, please learn to use them.
4. Detect the face
5. Detect the face landmarks
6. Transfrom the detected result to a 2-D numpy array by using the provided `shape2points` function from the *face_lib.py*.
7. Visualize the landmarks with the image

*Here is an example for facial landmarks extraction: http://dlib.net/face_landmark_detection.py.html (Note that this examples assumes multiple faces in the image. The images here only have a single sample, and no loop is required.)*



In [3]:
!pip3 install dlib

  Using cached dlib-19.24.2-cp311-cp311-macosx_13_0_arm64.whl


In [5]:
# Loading required libraries
import dlib
import numpy as np
import matplotlib.pyplot as plt
from face_lib import shape2points

In [7]:
# 1. Load example_img.jpg, using plt.imread()
image = plt.imread('example_img.jpg')

# 2. Visualize the image using plt.imshow()
plt.figure(figsize=(8, 8))

#showing the image
plt.imshow(image)

#plotting the figure
plt.show()

rectangles[[(241, 206) (562, 527)]]


In [159]:
# 3. Initializing face detector and shape predictor
detector = dlib.get_frontal_face_detector()
#the shape predictor is a neural network that is loaded from the .dat file
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 4. Detect face, return rectangles, each rectangle corresponds to one face.
# You need to fill the missing argument of this function

# dets = detector(..., 1)
dets = detector(image, 1)
print(dets)

# 5. Extract the shape of the face in the first rectangle (using the first element of the rectangles variable)
if len(dets) > 0:
    #using the shape predictor model
    shape = predictor(image, dets[0])
    print(shape)
else: 
    print('[NO FACES WERE DETECTED]')

# 6. Extract facial landmarks from shape by calling the shape2points() function.
from face_lib import shape2points
cords = shape2points(shape)
x_cords = []
y_cords = []
for i in range(len(cords)):
    x_cords.append(cords[i][0])
    y_cords.append(cords[i][1])
    
# 7. Visualize the landmarks (keypoints) by first plotting the image and then using plt.scatter() for the landmarks
plt.figure(figsize=(8, 8))
plt.imshow(image)

plt.scatter(x_cords, y_cords, s=10, color='blue')

plt.show()

### Task 1.2. Face normalization <font color='red'>(1 point)</font>
**Steps**:
1. Load the landmark position of a standard face model. We provide these positions in a csv file, and also the code block to read these positions.
2. Calculate the transformation between your detected landmarks position and the standard face model landmark positions using the [`skimage.transform.PolynomialTransform()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.PolynomialTransform) class and its *estimate()* methods. 
    1. Instantiate a PolynomialTransform object by calling transform.PolynomialTransform()
    2. Call its estimate() method to calculate the transformation between the two sets of points. The manual of this method can be found in the same page which introduced of this class. 
3. Transform the example image using the calculated transformation to register (map) the example image into a space of the standard face model. You can use the [`skimage.transform.warp()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.warp) function to perform this.
4. Crop the face from the registered face using the standard face model landmarks. The cropping function is provided in the *face_lib.py*, you can directly use it after importing. 
5. Also extract the face from the example image using your detected landmarks.
<!---Use **cropFace** to crop the facial image from the background. For details on the **cropFace** function, please read the function in **exercise1Lib.py**. Usually, we have to do the face registration to remove the subject characteristics. To do a  face registration, firstly you need a set of landmarks of a standard face model. Then you calculate the transformation between you detected landmarks and the standard landmarks. Last you apply the transformation on your example image/-->

In [42]:
# 1. Load the landmark position of the standard face model from a csv file and store them in standard_model
def load_landmarks_from_csv(
    file_name: str
) -> np.ndarray:
    """
    Reads landmarks from a csv file.
    Arguments
    file_name : A csv file with landmarks
    Returns
    numpy array with the landmarks
    """
    standard_model = np.zeros((68, 2))
    with open(file_name, "r") as f:
        for i, line in enumerate(f.readlines()):
            line_split = line.replace("\n", "").split(",")
            standard_model[i] = [float(value) for value in line_split]
    # Multiply with 500 (width) as the landmarks are normalized
    standard_model *= 500
    return standard_model

standard_model = load_landmarks_from_csv("mean.csv")

In [43]:
#checking out the standard_model landmarks csv
print(standard_model[:10])

[[ 39.61984569 169.61187056]
 [ 41.46097436 228.47768397]
 [ 48.39635546 287.82400836]
 [ 61.07075781 345.96080053]
 [ 84.34393177 400.17063181]
 [119.89469535 447.86625239]
 [162.83122626 488.53438125]
 [211.15914101 521.64500074]
 [265.88890103 530.40185563]
 [320.64814903 519.90962054]]


<!---**Then you need to calculate the transformation using [`skimage.transform.PolynomialTransform()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.PolynomialTransform)**. This class has several methods, expecially, you need to use the **estimate()** method to calculate the transformation between two sets of points. **Registering the example face** by transforming the example image into the standard model space, using [`skimage.transform.warp()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.warp) and you calculated transformation. **Cropping the original example image according to the detected landmarks, and the registered image according to the standard face mode. **-->

In [101]:
from skimage import transform
from face_lib import crop_face

# 2. Calculating the transorfmation between the two set of points 
# 2.1 Instantiating a PolynomialTransform() transform function
transform_function = transform.PolynomialTransform()

#was having issues in warping so checking types 
print(type(standard_model))
print(type(detected_landmarks))

#converting dected_landmark list to np array
detected_landmarks = np.array(detected_landmarks)


# 2.2 Calculating the transformation by calling the estimate() method.
#     You do not need to retuern any value after calling this methods,
#     because the transformation parameter is store in the object you instantiated after calling this methods.
transformation = transform_function.estimate(standard_model, detected_landmarks)
print(transformation)

# 3. Warp the example image using the transform.warp() function
warped_image = transform.warp(image, transform_function)

# 4. Crop the face from registered image using the provided crop_face function.
cropped_registered_face = crop_face(warped_image, standard_model)

# 5. Croping the face from the example image using detected landmarks.
cropped_example_face = crop_face(image, detected_landmarks)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
True


### Task 1.3. Display result <font color='red'>(2 points)</font>
Here you are asked to draw a figure with 3 x 2 subplots using [`matplotlib.pyplot.subplots()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html). Please read the manual of it and also the [`matplotlib.pyplot`](https://matplotlib.org/api/pyplot_api.html). 

Each of the subplots should have the following images:
* subplot [0, 0]: the original example image and detected landmarks.
* subplot [1, 0]: the face cropped from the example image. 
* subplot [2, 0]: the histogram of the face cropped from the example. 

*As an example, the three subplots are given above.* Then you need to implement:
* subplot [0, 1]: the registered face image.
* subplot [1, 1]: the face cropped from the registered face image.
* subplot [2, 1]: the histogram of the face cropped from the registered face image.



In [109]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import img_as_ubyte

# Constructing figure with 3x2 subplots
fig, ax = plt.subplots(3, 2, figsize=(15, 15))

# subplot [0, 0]: Show the original example image and detected landmarks
ax[0, 0].imshow(image, cmap='gray')  
ax[0, 0].scatter(detected_landmarks[:, 0], detected_landmarks[:, 1], c="red", s=6)
ax[0, 0].set_title('Original Image with Landmarks')

# subplot [1, 0]: Show the face cropped from the example image
ax[1, 0].imshow(cropped_example_face, cmap='gray')
ax[1, 0].set_title('Cropped Face from Example Image')

# subplot [2, 0]: Show the histogram of the face cropped from the example image
ax[2, 0].hist(cropped_example_face.ravel(), bins=256, color='blue', alpha=0.6)
ax[2, 0].set_title('Histogram of Cropped Face from Example Image')

# subplot [0, 1]: Show the registered image 
ax[0, 1].imshow(warped_image)
ax[0, 1].set_title('Registered Image')

# Place the model landmarks on the registered image 
ax[0, 1].scatter(standard_model[:, 0], standard_model[:, 1], c="red", s=6)
ax[0, 1].set_title('Registered Image with Landmarks')

# subplot [1, 1]: Show the face cropped from the registered image
ax[1, 1].imshow(cropped_registered_face, cmap='gray')
ax[1, 1].set_title('Cropped Face from Registered Image')

# subplot [2, 1]: Show the histogram of the face cropped from the registered image
ax[2, 1].hist(cropped_registered_face.ravel(), bins=256, color='blue', alpha=0.6)
ax[2, 1].set_title('Histogram of Cropped Face from Registered Image')

fig.tight_layout()
plt.show()


### Question 1. What is the difference between the cropped example image (subplot [1, 0]) and the cropped registered image (subplot [1, 1])? What is the mapping between the images called?
*Hints: Look at the two images very closely, the change is very subtle. The code may also provide hints for the transformation name.*

### Your answer:

Visually speaking both of the images are showing slight variation, for the cropped example image, the face looks at a slight angle (not fully straight), less scaled and not stretched on the other hand registered image is scaled, widdened(stretched) and has a straight orientation. Programmatically, it shows that one of the image was warped by using a polynomialTransform(general idea is that the space changed or mapped to polynomial) this affected the image's orientation. Afterwards, both (warped and unwarped images) were passed through a function called crop_face(). crop_face() takes 2 parameters, an image and a list of landmarks, based on which it crops the given image. Both of the images being discussed were passed to this function and had different landmark parameter due to which the cropping of the faces were different and thus resulting in differently cropped images.

The mapping in this case is Polynomial Transformation that guides on how the warped image would look in a new space. 

### Question 2. What is the purpose of the operation asked in the previous question, why is it done?

### Your answer:

The purposes for such operation are alignment of the objects and faces by the help of warping transformations, normalizing images (scale up or down) based on our requirements and computation power, feature extraction as relevant features can be highlighted, and having consistency in datasets. 

## Task 2. Feature extraction <font color='red'>(2 points)</font>
Here you are asked to extract LBP (Local Binary Pattern) features. LBP has been widely used in face recognition, facial expression recognition and texture classification.
Here you will use the [`skimage.feature.local_binary_pattern()`](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.local_binary_pattern) function to extract the LBP features. Steps are explained below.

**Steps**:
1. Define the LBP parameters. Before doing this, carefully read the documentation.
    1.  P: Number of neighbours, set P = 8
    2.  R: Radius of circle, set R = 1.0
    3.  LBP methods: set it as 'nri_uniform'

2. Extract the LBP face by calling the *skimage.feature.local_binary_pattern()* function

3. Calculate the histogram of the LBP face
    1. Caculate the histogram of the LBP face
    2. Normalize the histogram to make the histogram's sum one one (dividing each element by the sum of the histogram).

4. Visualize the result using two subplots.
    1. Draw the LBP face on the left side. Use the *cmap="gray"* parameter
    2. Draw the normalized histogram on the right side, but using *.stem()* function rather *.plot()* for this time.

In [121]:
from skimage.feature import local_binary_pattern
from skimage import color
# 1. Define the parameters to extract LBP features in (8, 1) neighborhood:

#    1.1 Set the number of neighbour P = 8
P = 8

#    1.2 Set the radius if circir R = 1.0
R = 1.0

#    2.3 Set the method as "nri_uniform"
method = 'nri_uniform'

# 2. Extract the LBP face using local_binary_pattern() with the defined parameters
# gray_image = color.rgb2gray(cropped_registered_face)
# gray_image = np.array(gray_image * 255, dtype=np.uint8)
# print(gray_image)
lbp_image = local_binary_pattern(cropped_registered_face, P, R, method)

# 3. Calculate the histogram of the LBP face (58 bins). Normalize the histogram's sum to one
hist, bins = np.histogram(lbp_image, bins=58, range=(0, 58))
hist = hist / np.sum(hist)

# 4. Visualize your result.
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plotting the LBP face on the left subplot
axes[0].imshow(lbp_image, cmap="gray")
axes[0].set_title("LBP Face")

# Plotting the normalized histogram on the right subplot using stem plot
bins = np.arange(58)
axes[1].stem(bins, hist, basefmt=" ", markerfmt="ro", linefmt="-b")
axes[1].set_title("Normalized LBP Histogram")


plt.tight_layout()
plt.show()

### Question 3. Why is feature extraction (e.g. LBP) used, why can't we simply classify the (raw) images in their original domain?

### Your answer:

The simple answer would be to save computational time and space, image data can contain millions of pixels and its very computational expensive to process every detail. It would take days to process or train a model with a RGB dataset that has millions of pixels per image, feature extraction helps in this regard it highlights the necessary features required for the modeling based on the problem and skips all other attributes. For instance, LBP ignores features like lighting, identity of the image or RGB color channels and focuses on the features that are relevant to facial expressions like edges. Techniques like Normalization and Standardization are some examples of feature extraction.

## Task 3. Feature Classification

For feature classification the SVM (Support Vector Machine) will be utilized. See the documentation for [`sklearn.svm.SVC()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). Mainly you will use its two methods: **fit()** to training the classifier and **predict()** to use the classifer for classification. There are following three subtasks you need to complete:
* Task 3.1. Load data
* Task 3.2. Train classifiers
* Task 3.3. Evaluate classifiers



### Task 3.1. Load data <font color='red'>(1.5 points)</font>
Firstly, you need to read *.mat* files using python. You can use the [`scipy.io.loadmat()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html#scipy.io.loadmat) function to read *.mat* file. In the provided **Task3_data.mat** file, different data are packed by different dictionaries which are list below:
* training_data
* testing_data
* training_class
* testing_class

For example if you would like to load the *training_data* from the file, you can use **scipy.io.loadmat('Task3_data.mat')['training_data']**

Print the shapes of data for a sanity check.



In [147]:
import scipy.io as sio

# Loading data using scipy.io.loadmat(), or sio.loadmat
mdata = sio.loadmat("Task3_data.mat")

# Load 'training_data'
sample_train = mdata["training_data"]

# Load 'testing_data'
sample_test = mdata["testing_data"]

# Load 'training_class'
sample_training_class = mdata['training_class']

#for (50,) dimension
sample_training_class = sample_training_class.ravel()

# Load 'testing_class'
sample_testing_class = mdata['testing_class']

#for (50,) dimension
sample_testing_class = sample_testing_class.ravel()

#Print the shapes from each of the data splits
print(sample_train.shape)
print(sample_test.shape)
print(sample_training_class.shape)
print(sample_testing_class.shape)

(50, 708)
(50, 708)
(50,)
(50,)


### Question 4. The training data (*sample_train*) has a shape of $(50, 708)$. What do the numbers refer to? What is 50 and what is 708?

### Your answer:

50 rows, 708 columns

### Task 3.3. Train SVM classifiers <font color='red'>(0.5 points)</font>
Use the **sklearn.svm** library to train Support Vector Machine (SVM) classifiers. The *sample_train* and *sample_test* matrices contain the calculated LBP-TOP features for the training and testing sets, respectively. The block size for LBP-TOP used for training and testing data are 2x2x1. The *label_test* group vector contains the class of samples: 1 = happy, 2 = sadness, corresponding to the rows of the training data matrices.

**Steps**:
1.  Construct an SVM classifier object using a linear kernel. See [`sklearn.svm.SVC()`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC).

2.  Use the [`fit()`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.fit) method and the *sample_train* and *label_train* to train your classifier.

In [148]:
from sklearn import svm
# 1. Initializing an SVM classifier, using linear kernel
svm_classifier = svm.SVC(kernel='linear')

# 2. Use the classifier to fit your training data
svm_classifier.fit(sample_train, sample_training_class)

SVC(kernel='linear')

### Task 3.3. Evaluate your classifiers <font color='red'>(2 points)</font>
**Steps**:
1. Use your trained classifer to classify the *sample_train* and *sample_test*, using the [`predict()`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict) method.
2. Calculate the classification accuracies when classifying the *sample_train* and *sample_test*, respectively. The correct class labels corresponding with the rows of the training and testing data matrices are in the variables *label_train* and *label_test*, respectively.
3. Calculate the confusion matrices when evaluating both the dataset by using [`sklearn.metrics.confusion_matrix()`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).


In [152]:
# 1. Predict the training data and testing data.
train_predictions = svm_classifier.predict(sample_train)
test_predictions = svm_classifier.predict(sample_test)

# 2. Calculate the accuracies of your prediction on training data and testing data, respectively.
#    2.1 Calculate the accuracy when classifying the training data 
from sklearn.metrics import accuracy_score

training_accuracy = accuracy_score(sample_training_class, train_predictions)
print("Training data accuracy: ")
print(training_accuracy)


#    2.2 Calculate the accuracy when classifying the test data
testing_accuracy = accuracy_score(sample_testing_class, test_predictions)
print("Testing data accuracy: ")
print(testing_accuracy)

# 3. Print/Draw the confusion matrix using sklearn.metrics.confusion_matrix
#    3.1 Calculate the confusion matrix when classifying the training data
from sklearn.metrics import confusion_matrix
confusion_matrix_train = confusion_matrix(sample_training_class, train_predictions)
print("Training data confusion matrix: ")
print(confusion_matrix_train)


#    3.2 Calculate the confusion matrix when classifying the testing data
confusion_matrix_test = confusion_matrix(sample_testing_class, test_predictions)
print("Testing data confusion matrix: ")
print(confusion_matrix_test)

Training data accuracy: 
0.88
Testing data accuracy: 
0.72
Training data confusion matrix: 
[[20  5]
 [ 1 24]]
Testing data confusion matrix: 
[[25  0]
 [14 11]]


### Question 5. Compare the classification results from train data and test data. Which one achieves a better accuracy? Why?

### Your answer:

The accuracy of the training data(.88) is better than that of the testing data(.72). The reasoning behind this is that the classification model was trained on the training data due to which it has showed overfitting, there might be some cases that the model learned on and they showed up again, hence, model predicted them accurately. On the other hand it is obvious that testing data would have slightly less accurate predictions since its new for the model. 

### Question 6. Look at the confusion matrix of test data. In which class did the classifier perform better, the happiness or sadness?

### Your answer:

Happiness